In [1]:
import pandas as pd
import numpy as np
import sys
import random
from sqlalchemy import create_engine

from snorkel.labeling import labeling_function

## Load Data

### DataFrame

In [2]:
## load the data by choosing a random 10% of the data
## is the data ordered? am I missing a large chunk due to my partial loading?
# p = 0.01
# responses = pd.read_csv('../data/moralmachine/SharedResponses.csv', skiprows=lambda i: i>0 and random.random() > p)

### SQL Engine

In [3]:
# access SQL DB with data
engine = create_engine("sqlite:///../data/moralmachine.db", echo=False)

In [4]:
# demo
# responses = pd.read_sql("SELECT * FROM sharedresponses ORDER BY RANDOM() LIMIT 100000", con=engine)
# responses.columns

In [5]:
# ## For viewing the possible values for each feature
# for feature in responses.columns:
#     print(feature)
#     print(responses[feature].unique())
#     print(responses[feature].unique().shape)

In [6]:
# ## What does a single user's session/response look like?
# responses_random = responses[responses['ScenarioTypeStrict'] == 'Random']
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     ## see a random user's session
# #     display(responses_random[responses_random['ExtendedSessionID'] == responses_random['ExtendedSessionID'].sample().values[0]])
    
#     ## see the user with nth most responses
#     n = 100
#     display(responses_random[responses_random['ResponseID'] == responses_random.groupby(by='ResponseID').size().sort_values(ascending=False).index[n]]) 

In [7]:
# ## group alternatives pairwise!
# # take only responses where both alternatives are present in the random sample - for a full sample, this step isn't necessary
# responses_grouped = responses.groupby(by='ResponseID').filter(lambda g: g.shape[0] < 2)
# responses_grouped.shape

## Data Exploration

In [8]:
# responses.describe()

## Snorkel Labeling
Using https://www.snorkel.org/use-cases/01-spam-tutorial

In [5]:
# query random sample of responses grouped by ResponseID; only take responses for which both instances are present
query = """
    --SELECT * FROM (
        SELECT * FROM sharedresponses
            WHERE ScenarioTypeStrict LIKE 'Random'
        ORDER BY RANDOM()
        LIMIT 100000
    --)
    --GROUP BY ResponseID, ExtendedSessionID, UserID
    --HAVING COUNT(ResponseID) > 1
"""
df = pd.read_sql(query, con=engine).groupby(by='ResponseID').filter(lambda g: g.shape[0] > 1).sort_values('ResponseID')
df

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
63330,26yNSSS6ksk3YYC5p,-1621715534_2488018525183497.0,2488018525183500,10,1,0,1,0,Rand,Random,...,0,0,1,0,0,0,0,1,0,0
1059,26yNSSS6ksk3YYC5p,-1621715534_2488018525183497.0,2488018525183500,10,0,0,0,2,Rand,Random,...,0,0,0,0,0,1,0,0,0,1
28321,2Ap8aMFF6gK4sXQJP,-1245267949_6649030353085017.0,6649030353085020,9,1,0,1,0,Rand,Random,...,1,1,0,0,0,0,0,0,0,0
34466,2Ap8aMFF6gK4sXQJP,-1245267949_6649030353085017.0,6649030353085020,9,0,0,0,2,Rand,Random,...,0,0,0,0,2,0,0,0,0,0
23049,2EneYQK7cQp6jTvvc,1511742945_3375907920.0,3375907920,1,0,0,0,1,Rand,Random,...,0,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12869,zw9XxQELwWcm3hj5b,-1302274529_4078192808437328.0,4078192808437330,13,1,1,0,2,Rand,Random,...,0,0,0,0,0,0,0,0,1,1
94724,zwaRNod8tRY35G9EK,-1090608537_4881549115412729.0,4881549115412730,2,0,0,0,1,Rand,Random,...,0,0,0,0,0,1,2,0,1,0
8120,zwaRNod8tRY35G9EK,-1090608537_4881549115412729.0,4881549115412730,2,1,0,1,0,Rand,Random,...,0,0,0,0,1,0,0,0,0,0
72414,zyBJHXKtmiCyMpkCJ,-593777922_6718076091886240.0,6718076091886240,1,0,0,1,0,Rand,Random,...,0,0,0,0,0,0,1,1,1,0


For comparison to Noothigattu et al., how many pairwise comparisons per voter in this sample dataset?

In [8]:
# number of pairwise comparisons per voter?
# = number of response IDs per voter
df.groupby('UserID')['ResponseID'].nunique().mean()

1.0083234244946493

Need to have both alternatives in the same tuple, marking them by whether or not they are saved by an intervention - specifically the factors:
> 'NumberOfCharacters', 'DiffNumberOfCharacters', 'Saved', 'Template', 'DescriptionShown',
'LeftHand', 'UserCountry3', 'Man', 'Woman', 'Pregnant', 'Stroller',
'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman',
'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive',
'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog',
'Cat'

In [10]:
df[["ResponseID","Intervention","Saved"]]

,ResponseID,Intervention,Saved
856770,2246cPHyxkJzG9JQg,0,0
578075,2246cPHyxkJzG9JQg,1,1
521255,224Xe9Dxn8tuYs4ZC,1,1
167862,224Xe9Dxn8tuYs4ZC,0,0
948266,226njsMQbN934Hkot,1,0
...,...,...,...
405525,zzstxiCateYBTbfW9,1,1
78675,zzttiiYjZmkH5eYD5,0,1
991362,zzttiiYjZmkH5eYD5,1,0
886398,zzuaDN7Ckh3sjWYAj,0,1


In [11]:
scenario_fields = [
    'ResponseID', 'Barrier', 'NumberOfCharacters', 'DiffNumberOFCharacters', 'Saved', 'DescriptionShown', 'LeftHand', 'Man', 'Woman', 'Pregnant', 'Stroller', \
    'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', \
    'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat'
]
intervention = df[df['Intervention'] == 1][scenario_fields].set_index('ResponseID')
no_intervention = df[df['Intervention'] == 0][scenario_fields].set_index('ResponseID')
df_joined = intervention.join(no_intervention, lsuffix='_int', rsuffix='_noint', how='inner')
df_joined = df[[col for col in df.columns if col not in scenario_fields or col == 'ResponseID']].set_index('ResponseID').join(df_joined)
df_joined['Intervened'] = (df_joined['Saved_int'] == 1).astype(int)
df_joined = df_joined.drop(axis='columns', labels=['Saved_{}'.format(s) for s in ['int', 'noint']])
df_joined

,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,CrossingSignal,AttributeLevel,ScenarioTypeStrict,ScenarioType,DefaultChoice,...,Criminal_noint,MaleExecutive_noint,FemaleExecutive_noint,FemaleAthlete_noint,MaleAthlete_noint,FemaleDoctor_noint,MaleDoctor_noint,Dog_noint,Cat_noint,Intervened
ResponseID,,,,,,,,,,,,,,,,,,,,,
2246cPHyxkJzG9JQg,-122965818_2093651351,2093651351,4,0,0,2,Rand,Random,Random,,...,0,0,1,0,0,0,0,0,0,1
2246cPHyxkJzG9JQg,-122965818_2093651351,2093651351,4,1,0,0,Rand,Random,Random,,...,0,0,1,0,0,0,0,0,0,1
224Xe9Dxn8tuYs4ZC,1197038880_3798649535722139.0,3798649535722140,9,1,0,0,Rand,Random,Random,,...,0,0,0,0,0,0,0,1,0,1
224Xe9Dxn8tuYs4ZC,1197038880_3798649535722139.0,3798649535722140,9,0,0,0,Rand,Random,Random,,...,0,0,0,0,0,0,0,1,0,1
226njsMQbN934Hkot,-874887960_4265756189384999.0,4265756189385000,1,1,0,2,Rand,Random,Random,,...,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzstxiCateYBTbfW9,1411367944_3710906850.0,3710906850,7,1,0,0,Rand,Random,Random,,...,0,0,0,0,0,0,0,0,1,1
zzttiiYjZmkH5eYD5,-890617025_1351509525401422.0,1351509525401420,13,0,0,1,Rand,Random,Random,,...,0,0,0,0,0,0,1,0,0,0
zzttiiYjZmkH5eYD5,-890617025_1351509525401422.0,1351509525401420,13,1,0,0,Rand,Random,Random,,...,0,0,0,0,0,0,1,0,0,0


A standard train test split for testing:

In [12]:
from sklearn.model_selection import train_test_split

X = df_joined.drop(labels=["Intervened"], axis='columns', inplace=False)
y = df_joined["Intervened"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
X_test, X_dev, y_test, y_dev = train_test_split(X_test, y_test, test_size=0.2, random_state=1)

Some sample labeling functions:

In [13]:
sys.path.insert(1, '../heuristics')
import labeling_functions
import utils
import importlib
importlib.reload(labeling_functions)
importlib.reload(utils)

from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis


lfs = [
    labeling_functions.doctors,
    labeling_functions.utilitarian,
    labeling_functions.utilitarian_anthro,
    labeling_functions.inaction,
    labeling_functions.pedestrians,
    labeling_functions.females,
    labeling_functions.fitness,
    labeling_functions.status,
    labeling_functions.legal,
    labeling_functions.illegal,
    labeling_functions.youth,
    labeling_functions.criminals,
    labeling_functions.homeless,
    labeling_functions.pets,
    labeling_functions.spare_strollers,
    labeling_functions.spare_girl,
    labeling_functions.spare_boy,
    labeling_functions.spare_pregnant
]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=X_train)
L_dev = applier.apply(df=X_dev)
L_valid = applier.apply(df=X_val)
LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=y_dev.values).sort_values("Correct", ascending=False)

/Users/steed/miniconda3/envs/hmm/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 26831/26831 [00:27<00:00, 977.33it/s] 


,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
utilitarian_anthro,2,"[0, 1]",0.802922,0.802922,0.742397,3761,1625,0.698292
utilitarian,1,"[0, 1]",0.802326,0.802326,0.743739,3705,1677,0.688406
inaction,3,[0],1.000000,0.999553,0.921139,3224,3484,0.480620
pedestrians,4,"[0, 1]",0.880739,0.880739,0.825134,2790,3118,0.472241
youth,10,"[0, 1]",0.569171,0.569171,0.528623,2563,1255,0.671294
females,5,"[0, 1]",0.588551,0.588551,0.546064,2401,1547,0.608156
fitness,6,"[0, 1]",0.593023,0.593023,0.558587,1994,1984,0.501257
doctors,0,"[0, 1]",0.402504,0.402504,0.380888,1644,1056,0.608889
status,7,"[0, 1]",0.392516,0.392516,0.370751,1532,1101,0.581846
spare_boy,16,"[0, 1]",0.239267,0.239267,0.227042,1029,576,0.641121


## Aggregation

Recall that there are no true labels for this problem - really, we're just measuring similarity of the heuristic labels to real voter's responses. 

**Baseline**: majority label voting.

In [14]:
from snorkel.labeling import MajorityLabelVoter

model_majority = MajorityLabelVoter()
preds_train = model_majority.predict(L=L_train)
preds_train

array([0, 1, 1, ..., 0, 0, 0])

Snorkel label model.

In [15]:
from snorkel.labeling import LabelModel

# cardinality is num classes
model_label = LabelModel(cardinality=2, verbose=True)
model_label.fit(L_train=L_train, n_epochs=500, lr=.001, log_freq=100, seed=1)

In [16]:
for model in [model_majority, model_label]:
    acc = model.score(L=L_valid, Y=y_val.values, tie_break_policy="random")["accuracy"]
    print(f"{str(model)} {'Vote Accuracy:':<25} {acc*100:.1f}")

MajorityLabelVoter() Vote Accuracy:            64.0
LabelModel() Vote Accuracy:            66.9


In [17]:
from snorkel.analysis import get_label_buckets

probs_dev = model_label.predict_proba(L=L_dev)
preds_dev = probs_dev >= 0.5
buckets = get_label_buckets(y_dev.values, preds_dev[:, 1])

df_fn_dev = X_dev.iloc[buckets[(1, 0)]]
df_fn_dev["probability"] = probs_dev[buckets[(1, 0)], 1]

df_fn_dev.sample(5, random_state=3)

/Users/steed/miniconda3/envs/hmm/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,CrossingSignal,AttributeLevel,ScenarioTypeStrict,ScenarioType,DefaultChoice,...,Criminal_noint,MaleExecutive_noint,FemaleExecutive_noint,FemaleAthlete_noint,MaleAthlete_noint,FemaleDoctor_noint,MaleDoctor_noint,Dog_noint,Cat_noint,probability
ResponseID,,,,,,,,,,,,,,,,,,,,,
ct4iJ5MFkbzFzhcye,-622927709_9311317572192356.0,9311317572192360,1,1,0,2,Rand,Random,Random,,...,0,1,1,0,0,1,1,0,0,2.418978e-12
6PW4rs2Zojeqy6SZY,482684800_2297977377.0,2297977377,11,1,0,0,Rand,Random,Random,,...,0,1,0,1,0,0,1,1,1,5.787899e-06
JYvXfgwccRMwBe7h4,-179661022_1811539112,1811539112,1,1,0,0,Rand,Random,Random,,...,1,0,0,0,0,0,0,0,1,5.426635e-06
85uRZorjyMZNN4EqT,1469845862_5850049562178248.0,5850049562178250,13,0,0,0,Rand,Random,Random,,...,0,1,0,0,0,0,0,0,0,2.582897e-14
3Ri7PcBL4ehBFox8H,-1560017716_7234197248835528.0,7234197248835530,6,1,0,0,Rand,Random,Random,,...,0,0,0,0,1,0,0,1,0,1.150624e-05


Filter out unlabeled data points.

## Classification

https://www.snorkel.org/use-cases/01-spam-tutorial#5-training-a-classifier

### Featurization

In [18]:
# include user countries and other metadata or not? decided not - experts writing LFs are trying to decide most moral response, not predict what an individual from a certain background would do
# but if it helps generalization, technically useful... try both?
features = [
    "PedPed", "CrossingSignal", 'NumberOfCharacters_int', 'Man_int', 'Woman_int',
    'Pregnant_int', 'Stroller_int', 'OldMan_int', 'OldWoman_int', 'Boy_int',
    'Girl_int', 'Homeless_int', 'LargeWoman_int', 'LargeMan_int',
    'Criminal_int', 'MaleExecutive_int', 'FemaleExecutive_int',
    'FemaleAthlete_int', 'MaleAthlete_int', 'FemaleDoctor_int', 'Barrier_int',
    'MaleDoctor_int', 'Dog_int', 'Cat_int', 'Barrier_noint', 'NumberOfCharacters_noint',
    'Man_noint', 'Woman_noint', 'Pregnant_noint', 'Stroller_noint', 'OldMan_noint', 'OldWoman_noint',
    'Boy_noint', 'Girl_noint', 'Homeless_noint', 'LargeWoman_noint',
    'LargeMan_noint', 'Criminal_noint', 'MaleExecutive_noint',
    'FemaleExecutive_noint', 'FemaleAthlete_noint', 'MaleAthlete_noint',
    'FemaleDoctor_noint', 'MaleDoctor_noint', 'Dog_noint', 'Cat_noint', 
#     "Template", "UserCountry3", 'DescriptionShown_int', 'LeftHand_int', 'DescriptionShown_noint', 'LeftHand_noint'
]
cat_features = [
#     "Template", "UserCountry3"
]
num_features = [
    "PedPed", "CrossingSignal", "NumberOfCharacters_int", "NumberOfCharacters_noint", "Man_int", "Man_noint", "Woman_int", "Woman_noint", 
    "Pregnant_int", "Pregnant_noint", "Stroller_int", "Stroller_noint", "OldMan_int", "OldMan_noint", "OldWoman_int", "OldWoman_noint", "Boy_int", "Boy_noint",
    "LargeMan_noint", "LargeMan_int", "Criminal_int", "Criminal_noint", "MaleExecutive_int", "MaleExecutive_noint", "FemaleExecutive_int",
    "FemaleExecutive_noint", "Girl_int", "Girl_noint", "LargeWoman_int", "LargeWoman_noint", "FemaleAthlete_int", "FemaleAthlete_noint",
    "MaleAthlete_int", "MaleAthlete_noint", "FemaleDoctor_int", "FemaleDoctor_noint", "MaleDoctor_int", "MaleDoctor_noint", "Dog_int",
    "Dog_noint", "Cat_int", "Cat_noint", "Homeless_int", "Barrier_noint", "Homeless_noint", "Barrier_int"
#   'DescriptionShown_int', 'LeftHand_int', 'DescriptionShown_noint', 'LeftHand_noint'
]
# are there any nan?
X_train[num_features].describe()

,PedPed,CrossingSignal,Barrier_noint,Barrier_int
count,107322.000000,107322.000000,107322.000000,107322.000000
mean,0.119202,0.757235,0.400291,0.480507
std,0.324028,0.844620,0.489959,0.499622
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,2.000000,1.000000,1.000000
max,1.000000,2.000000,1.000000,1.000000


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

def transform_X(X):
    X = X[features]
    # convert to numeric, changing literals to NaN
    for f in num_features:
        X[f] = pd.to_numeric(X[f], errors='coerce')
    return X

X_train = transform_X(X_train)
X_dev = transform_X(X_dev)
X_val = transform_X(X_val)
X_test = transform_X(X_test)

# https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('oh_enc', OneHotEncoder(handle_unknown='ignore'))
])
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ]
)
clf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100))
#     ('classifier', LogisticRegression(C=.001))
])

/Users/steed/miniconda3/envs/hmm/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


Using just the labels (no label model):

In [20]:
from snorkel.analysis import metric_score

def test_accuracy(clf):
    preds_test_dev = np.round(clf.predict(X_test))
    test_acc = metric_score(golds=y_test, preds=preds_test_dev, metric="accuracy")
    print(f"Test Accuracy: {test_acc * 100:.1f}%")
    
clf.fit(X=X_train, y=y_train)
test_accuracy(clf)

Test Accuracy: 82.9%


Using the label model:

In [21]:
from snorkel.labeling import filter_unlabeled_dataframe

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=X_train, y=model_label.predict_proba(L=L_train), L=L_train
)
df_train_filtered

,PedPed,CrossingSignal,NumberOfCharacters_int,Man_int,Woman_int,Pregnant_int,Stroller_int,OldMan_int,OldWoman_int,Boy_int,...,LargeMan_noint,Criminal_noint,MaleExecutive_noint,FemaleExecutive_noint,FemaleAthlete_noint,MaleAthlete_noint,FemaleDoctor_noint,MaleDoctor_noint,Dog_noint,Cat_noint
ResponseID,,,,,,,,,,,,,,,,,,,,,
amAqXftPdTjeq2BqK,1,2,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,2,1,0
y74L4Em6gR8BwaRez,0,0,3,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
W6Qc5xfozrnebhPRR,0,1,4,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
327AQ3oN82RnGCdb3,0,1,3,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,1,0,0
SXf9eXPHAWZdoB5wG,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47wdS2HrqS4Y8Ayg2,0,0,2,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PZ4zjX3dZAKxhKmgD,0,0,3,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
BuTuepkymLAgEcRi8,0,2,4,0,0,0,0,2,0,0,...,0,0,0,0,1,0,0,0,0,0


In [22]:
from snorkel.utils import probs_to_preds

preds_train_filtered = probs_to_preds(probs=probs_train_filtered)
clf.fit(X=df_train_filtered, y=preds_train_filtered)
test_accuracy(clf)

Test Accuracy: 66.7%
